# **MÓDULO 35 - Cross Validation**

Nesta tarefa, você trabalhará com uma base de dados que contém informações sobre variáveis ambientais coletadas para a detecção de incêndios. O objetivo é utilizar técnicas de validação cruzada (cross-validation) para avaliar a performance de um modelo de classificação na previsão da ocorrência de um incêndio com base nas variáveis fornecidas.


Descrição da Base de Dados
A base de dados contém as seguintes variáveis:

Unnamed:0: Índice (não é uma variável útil para o modelo)

UTC: Tempo em Segundos UTC

Temperature[C]: Temperatura do Ar (em graus Celsius)

Humidity[%]: Umidade do Ar (em porcentagem)

TVOC[ppb]: Total de Compostos Orgânicos Voláteis (medido em partes por bilhão)

eCO2[ppm]: Concentração equivalente de CO2 (medido em partes por milhão)

Raw H2: Hidrogênio molecular bruto, não compensado

Raw Ethanol: Etanol gasoso bruto

Pressure[hPA]: Pressão do Ar (em hectopascais)

PM1.0: Material particulado de tamanho < 1,0 µm

PM2.5: Material particulado de tamanho >1,0 µm e < 2,5 µm

NC0.5: Concentração numérica de material particulado de tamanho < 0,5 µm

NC1.0: Concentração numérica de material particulado de tamanho 0,5 µm < 1,0 µm

NC2.5: Concentração numérica de material particulado de tamanho 1,0 µm < 2,5 µm

CNT: Contador de amostras


E a variável alvo:

Fire Alarm: Indicador binário de incêndio (1 se houver incêndio, 0 caso contrário)

O objetivo desta tarefa é aplicar a técnica de validação cruzada (cross-validation) para avaliar a performance de um modelo de classificação. A validação cruzada ajudará a garantir que o modelo seja avaliado de maneira robusta e generalize bem para dados não vistos.

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# 1 - Carregue a base de dados, verifique os tipos de dados e também se há presença de dados faltantes ou nulos.

In [2]:
# ETAPA 1 - CARREGAMENTO E INSPEÇÃO INICIAL DOS DADOS

# IMPORTAÇÃO DE BIBLIOTECAS
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# CARREGAMENTO DA BASE DE DADOS
caminho_arquivo = "Cientista de dados M35 - smoke_detection_iot.csv"
df_fogo = pd.read_csv(caminho_arquivo)

# VISUALIZAÇÃO INICIAL
print("Prévia da base de dados:")
print(df_fogo.head())

# TIPOS DE DADOS
print("\nTipos de dados por coluna:")
print(df_fogo.dtypes)

# VERIFICAÇÃO DE DADOS NULOS
print("\nContagem de valores nulos por coluna:")
print(df_fogo.isnull().sum())


Prévia da base de dados:
   Unnamed: 0         UTC  Temperature[C]  Humidity[%]  TVOC[ppb]  eCO2[ppm]  \
0           0  1654733331          20.000        57.36          0        400   
1           1  1654733332          20.015        56.67          0        400   
2           2  1654733333          20.029        55.96          0        400   
3           3  1654733334          20.044        55.28          0        400   
4           4  1654733335          20.059        54.69          0        400   

   Raw H2  Raw Ethanol  Pressure[hPa]  PM1.0  PM2.5  NC0.5  NC1.0  NC2.5  CNT  \
0   12306        18520        939.735    0.0    0.0    0.0    0.0    0.0    0   
1   12345        18651        939.744    0.0    0.0    0.0    0.0    0.0    1   
2   12374        18764        939.738    0.0    0.0    0.0    0.0    0.0    2   
3   12390        18849        939.736    0.0    0.0    0.0    0.0    0.0    3   
4   12403        18921        939.744    0.0    0.0    0.0    0.0    0.0    4   

   Fire

In [3]:
# ETAPA 2 - AJUSTES INICIAIS DE NOMES DE COLUNAS

# RENOMEANDO A COLUNA ALVO
df_fogo.rename(columns={"Fire Alarm": "Fire_Alarm"}, inplace=True)

# VERIFICAÇÃO DOS NOVOS NOMES DE COLUNAS
print(df_fogo.columns)


Index(['Unnamed: 0', 'UTC', 'Temperature[C]', 'Humidity[%]', 'TVOC[ppb]',
       'eCO2[ppm]', 'Raw H2', 'Raw Ethanol', 'Pressure[hPa]', 'PM1.0', 'PM2.5',
       'NC0.5', 'NC1.0', 'NC2.5', 'CNT', 'Fire_Alarm'],
      dtype='object')


Para a coluna Fire Alarm, por conta do espaçamento talvez seja util renomear o nome da coluna utilizando:

df.rename(columns={'Fire Alarm': 'Fire_Alarm'}, inplace=True)

# 2 - Para essa base, onde você realizará as previsões de fire alarm, qual modelo de machine learning você aplicará? Justifique.




Eu vou aplicar o modelo de **Regressão Logística** para prever a variável 'Fire_Alarm'.
Isso porque se trata de um problema de classificação binária (0 = sem incêndio, 1 = incêndio), que é exatamente o tipo de tarefa para o qual a Regressão Logística foi desenvolvida. As variáveis preditoras são todas numéricas e contínuas (temperatura, umidade, pressão, gases, partículas), o que se encaixa muito bem nas premissas do modelo. Além disso, a Regressão Logística é um algoritmo simples, rápido, estável e interpretável, permitindo analisar o impacto de cada variável na probabilidade de ocorrência de incêndio. Por fim, como o foco do módulo é validação cruzada, esse modelo serve como um baseline robusto para avaliar a performance e a capacidade de generalização nas diferentes folds da cross-validation.


# 3 - Separe a base em Y e X e já rode a instância do modelo que você utilizará.

In [4]:
# ETAPA 3 - DEFINIÇÃO DE X E Y E INSTÂNCIA DO MODELO

# DEFINIÇÃO DA VARIÁVEL ALVO (y)
y = df_fogo["Fire_Alarm"]

# DEFINIÇÃO DAS VARIÁVEIS PREDITORAS (X)
colunas_features = [
    col
    for col in df_fogo.columns
    if col not in ["Fire_Alarm", "Unnamed: 0", "Unnamed:0"]
]
X = df_fogo[colunas_features]

print("Colunas usadas em X:")
print(colunas_features)

# INSTÂNCIA DO MODELO DE REGRESSÃO LOGÍSTICA
modelo_logistico = LogisticRegression(max_iter=1000)

print("\nModelo instanciado:")
print(modelo_logistico)


Colunas usadas em X:
['UTC', 'Temperature[C]', 'Humidity[%]', 'TVOC[ppb]', 'eCO2[ppm]', 'Raw H2', 'Raw Ethanol', 'Pressure[hPa]', 'PM1.0', 'PM2.5', 'NC0.5', 'NC1.0', 'NC2.5', 'CNT']

Modelo instanciado:
LogisticRegression(max_iter=1000)


# 4 - Defina o número de Folds e rode o modelo com a validação cruzada.

In [5]:
# ETAPA 4 - DEFINIÇÃO DOS FOLDS E VALIDAÇÃO CRUZADA

# DEFINIÇÃO DO ESQUEMA DE KFOLD
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# EXECUÇÃO DA VALIDAÇÃO CRUZADA COM ACURÁCIA
scores_cv = cross_val_score(
    modelo_logistico,
    X,
    y,
    cv=kfold,
    scoring="accuracy"
)

# RESULTADOS DA VALIDAÇÃO CRUZADA
print("Scores de acurácia em cada fold:", scores_cv)
print("Média de acurácia:", scores_cv.mean())
print("Desvio padrão da acurácia:", scores_cv.std())


Scores de acurácia em cada fold: [0.98555006 0.98626856 0.98626856 0.97700782 0.9776465 ]
Média de acurácia: 0.9825482995369631
Desvio padrão da acurácia: 0.0042758810739395


# 5 - Avalie a pontuação de cada modelo e ao final a validação final da média.

In [6]:
# ETAPA 5 - AVALIAÇÃO DAS PONTUAÇÕES POR FOLD E DA MÉDIA FINAL

# IMPRESSÃO DETALHADA DOS SCORES POR FOLD
for i, score in enumerate(scores_cv, start=1):
    print(f"Fold {i} - Acurácia: {score:.4f}")

# CÁLCULO DA MÉDIA E DESVIO PADRÃO
media_acuracia = scores_cv.mean()
desvio_acuracia = scores_cv.std()

print("\nAcurácia média (cross-validation):", round(media_acuracia, 4))
print("Desvio padrão da acurácia:", round(desvio_acuracia, 4))


Fold 1 - Acurácia: 0.9856
Fold 2 - Acurácia: 0.9863
Fold 3 - Acurácia: 0.9863
Fold 4 - Acurácia: 0.9770
Fold 5 - Acurácia: 0.9776

Acurácia média (cross-validation): 0.9825
Desvio padrão da acurácia: 0.0043


Na validação cruzada utilizando K-Fold com 5 folds, o modelo de Regressão Logística apresentou as seguintes acurácias em cada partição: 0.9856, 0.9863, 0.9863, 0.9770 e 0.9776. Observa-se que todos os valores são elevados e relativamente próximos entre si, sem grandes variações de desempenho entre os folds.
A acurácia média obtida foi de aproximadamente 0.9825, indicando que o modelo acerta cerca de 98% das observações na tarefa de previsão da variável Fire_Alarm. Além disso, o desvio padrão da acurácia foi de 0.0043, um valor bastante baixo, o que mostra que o modelo é estável e mantém desempenho consistente independentemente da divisão dos dados.
Com base nesses resultados, conclui-se que o modelo apresenta boa capacidade de generalização e a validação cruzada confirma que ele não depende de uma única amostra específica de treino e teste. A média de acurácia obtida na cross-validation pode ser considerada uma estimativa confiável da performance do modelo em novos dados de detecção de incêndio.